<a href="https://colab.research.google.com/github/GabrielSandoval22/Climate_system_modeling/blob/main/CDR_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create classes for each stock

Need to figure out what specific variables to initialize

Afforestable Land Stock (works)

In [ ]:
class AfforestableLand:
    def __init__(self, cdr_start_year, target_afforested_land, hectares_per_mha,
                 land_still_to_be_afforested, years_to_secure_land_for_afforestation,
                 years_to_plant_land_committed_to_afforestation):
        """
        Initializes the Afforestable Land stock.
        Inputs can be either constants or functions returning dynamic values.
        """
        self.cdr_start_year = cdr_start_year
        self.target_afforested_land = target_afforested_land  # Can be a constant or function
        self.hectares_per_mha = hectares_per_mha
        self.land_still_to_be_afforested = land_still_to_be_afforested
        self.years_to_secure_land_for_afforestation = years_to_secure_land_for_afforestation
        self.years_to_plant_land_committed_to_afforestation = years_to_plant_land_committed_to_afforestation
        self.afforestable_land_val = 0  # Initial stock value

    def get_target_afforested_land(self, time):
        """Returns either a constant value or computes it dynamically."""
        # Check if target_afforested_land is a callable function
        if callable(self.target_afforested_land):
            return self.target_afforested_land(time)
        return self.target_afforested_land  # Return as constant if not a function

    def committing_land_to_afforestation(self, time):
        """Calculates the inflow to afforestation stock."""
        if time < self.cdr_start_year:
            return 0
        return min(
            self.get_target_afforested_land(time) * self.hectares_per_mha,
            self.land_still_to_be_afforested
        ) / self.years_to_secure_land_for_afforestation

    def land_area_per_year_of_afforestation(self):
        """Calculates the outflow from the afforestation stock."""
        return self.afforestable_land_val / self.years_to_plant_land_committed_to_afforestation if self.years_to_plant_land_committed_to_afforestation > 0 else 0

    def update_stock(self, time):
        """Updates the stock value dynamically each year."""
        inflow = self.committing_land_to_afforestation(time)
        outflow = self.land_area_per_year_of_afforestation()
        self.afforestable_land_val += inflow - outflow

    def get_stock_value(self):
        """Returns the current afforestable land."""
        return self.afforestable_land_val

# Example Usage:
# Create an instance of AfforestableLand
afforestable = AfforestableLand(cdr_start_year=2030, target_afforested_land=10, hectares_per_mha=1e6,
                                land_still_to_be_afforested=500, years_to_secure_land_for_afforestation=5, years_to_plant_land_committed_to_afforestation=10)

# Simulate for 10 years
for year in range(2030, 2040):
    afforestable.update_stock(year)  # Update afforestable land stock first
    print(f"Year {year}: Afforestable Land = {afforestable.get_stock_value():.2f} hectares")


Year 2030: Afforestable Land = 100.00 hectares
Year 2031: Afforestable Land = 190.00 hectares
Year 2032: Afforestable Land = 271.00 hectares
Year 2033: Afforestable Land = 343.90 hectares
Year 2034: Afforestable Land = 409.51 hectares
Year 2035: Afforestable Land = 468.56 hectares
Year 2036: Afforestable Land = 521.70 hectares
Year 2037: Afforestable Land = 569.53 hectares
Year 2038: Afforestable Land = 612.58 hectares
Year 2039: Afforestable Land = 651.32 hectares


Afforested Land Stock (works)

In [ ]:
class AfforestedLand:
    def __init__(self, afforestable_land):
        """
        Initializes the Afforested Land stock.
        Inputs:
        - afforestable_land: An instance of the AfforestableLand class
        """
        self.afforestable_land = afforestable_land  # Link to AfforestableLand
        self.afforested_land_val = 0  # Initial stock value

    def update_stock(self, time):
        """Updates afforested land by integrating the inflow over time."""
        inflow = self.afforestable_land.land_area_per_year_of_afforestation() # should this include time, or just self?
        self.afforested_land_val += inflow  # Integrating the inflow over time

    def get_stock_value(self):
        """Returns the total afforested land."""
        return self.afforested_land_val

# Example Usage:
# Create an instance of AfforestableLand
afforestable = AfforestableLand(cdr_start_year=2030, target_afforested_land=10, hectares_per_mha=100,
                                land_still_to_be_afforested=500, years_to_secure_land_for_afforestation=5, years_to_plant_land_committed_to_afforestation=10)

# Create an instance of AfforestedLand, linking it to the afforestable instance
afforested = AfforestedLand(afforestable)

# Simulate for 10 years
for year in range(2030, 2040):
    afforestable.update_stock(year)  # Update afforestable land stock first
    afforested.update_stock(year)  # Then update afforested land stock
    print(f"Year {year}: Afforested Land = {afforested.get_stock_value():.2f} hectares")

Year 2030: Afforested Land = 10.00 hectares
Year 2031: Afforested Land = 29.00 hectares
Year 2032: Afforested Land = 56.10 hectares
Year 2033: Afforested Land = 90.49 hectares
Year 2034: Afforested Land = 131.44 hectares
Year 2035: Afforested Land = 178.30 hectares
Year 2036: Afforested Land = 230.47 hectares
Year 2037: Afforested Land = 287.42 hectares
Year 2038: Afforested Land = 348.68 hectares
Year 2039: Afforested Land = 413.81 hectares


Carbon Capture Experience Stock (works)

In [ ]:
class CarbonCaptureExperience:
    def __init__(self, ccs_experience_init_multiplier, ccs_reference_experience,
                 total_ccs_capture_by_fuel_tons_dict, tons_co2_per_gton_co2):
        """
        Initializes the Carbon Capture Experience stock.
        Inputs can either be constants or functions returning dynamic values.
        - total_ccs_capture_by_fuel_tons_dict: Dictionary mapping fuel types to their CCS capture in tons.
        """
        self.ccs_experience_init_multiplier = ccs_experience_init_multiplier
        self.ccs_reference_experience = ccs_reference_experience
        self.total_ccs_capture_by_fuel_tons_dict = total_ccs_capture_by_fuel_tons_dict  # Dictionary
        self.tons_co2_per_gton_co2 = tons_co2_per_gton_co2  # Constant

        # Initial stock value as per the integral equation
        self.carbon_capture_experience = self.ccs_experience_init_multiplier * self.ccs_reference_experience

    def total_ccs_capture_by_fuel_gt(self):
        """Computes Total CCS capture by fuel in Gt for all fuels."""
        return sum(tons / self.tons_co2_per_gton_co2 for tons in self.total_ccs_capture_by_fuel_tons_dict.values())

    def total_ccs_capture(self):
        """Computes the total CCS capture as the sum over all fuel types."""
        return self.total_ccs_capture_by_fuel_gt()

    def update_stock(self, time):
        """Updates the Carbon Capture Experience stock based on Total CCS Capture."""
        inflow = self.total_ccs_capture()
        self.carbon_capture_experience += inflow  # Integrating Total CCS Capture over time

    def get_stock_value(self):
        """Returns the current Carbon Capture Experience."""
        return self.carbon_capture_experience

# Example usage
# Define constants
ccs_experience_init_multiplier = 1  # Constant from Enroads
ccs_reference_experience = 2  # Constant from Enroads
tons_co2_per_gton_co2 = 1e9  # 1 Gt = 1 billion tons

# Define initial CCS capture by fuel type (in tons of CO2)
initial_ccs_capture_by_fuel = {
    "coal": 5e7,  # 50 million tons
    "oil": 3e7,   # 30 million tons
    "gas": 2e7    # 20 million tons
}

# Create an instance of CarbonCaptureExperience
carbon_capture_experience = CarbonCaptureExperience(
    ccs_experience_init_multiplier,
    ccs_reference_experience,
    initial_ccs_capture_by_fuel,
    tons_co2_per_gton_co2
)

# Check initial stock value
print(f"Initial Carbon Capture Experience: {carbon_capture_experience.get_stock_value()}")

# Simulate updates over 5 years
for year in range(2025, 2030):
    print(f"\nYear: {year}")

    # Example: CCS capture increases 5% per year for each fuel
    for fuel in initial_ccs_capture_by_fuel:
        carbon_capture_experience.total_ccs_capture_by_fuel_tons_dict[fuel] *= 1.05

    # Update the stock
    carbon_capture_experience.update_stock(year)

    # Print the updated stock value
    print(f"Carbon Capture Experience: {carbon_capture_experience.get_stock_value()}")

Initial Carbon Capture Experience: 2

Year: 2025
Carbon Capture Experience: 2.105

Year: 2026
Carbon Capture Experience: 2.21525

Year: 2027
Carbon Capture Experience: 2.3310125

Year: 2028
Carbon Capture Experience: 2.4525631249999997

Year: 2029
Carbon Capture Experience: 2.58019128125


Carbon Capture Experience Stock (updated 4/7)

In [5]:
class CarbonCaptureExperience:
    def __init__(self, ccs_experience_init_multiplier, ccs_reference_experience,
                 total_ccs_capture_by_fuel_tons_dict, tons_co2_per_gton_co2,
                 elec_desired_utilization_with_ccs, policy_adjustment_to_utilization,
                 ccs_fraction_in_elec_capacity, utilization_of_ccs_equipment,
                 elec_energy_supply_capacity):
        """
        Initializes the Carbon Capture Experience stock.

        - total_ccs_capture_by_fuel_tons_dict: {fuel: tons}
        - elec_desired_utilization_with_ccs, policy_adjustment_to_utilization,
          ccs_fraction_in_elec_capacity, utilization_of_ccs_equipment,
          elec_energy_supply_capacity: all should be dicts with fuel/paths as keys
        """
        self.ccs_experience_init_multiplier = ccs_experience_init_multiplier
        self.ccs_reference_experience = ccs_reference_experience
        self.total_ccs_capture_by_fuel_tons_dict = total_ccs_capture_by_fuel_tons_dict
        self.tons_co2_per_gton_co2 = tons_co2_per_gton_co2

        # These are needed for the desired output
        self.elec_desired_utilization_with_ccs = elec_desired_utilization_with_ccs
        self.policy_adjustment_to_utilization = policy_adjustment_to_utilization
        self.ccs_fraction_in_elec_capacity = ccs_fraction_in_elec_capacity
        self.utilization_of_ccs_equipment = utilization_of_ccs_equipment
        self.elec_energy_supply_capacity = elec_energy_supply_capacity

        # Stock initialization
        self.carbon_capture_experience = self.ccs_experience_init_multiplier * self.ccs_reference_experience

    def total_ccs_capture_by_fuel_gt(self):
        return sum(tons / self.tons_co2_per_gton_co2 for tons in self.total_ccs_capture_by_fuel_tons_dict.values())

    def total_ccs_capture(self):
        return self.total_ccs_capture_by_fuel_gt()

    def update_stock(self, time):
        inflow = self.total_ccs_capture()
        self.carbon_capture_experience += inflow

    def get_stock_value(self):
        return self.carbon_capture_experience

    def get_electricity_production_desired_with_ccs(self):
        """
        Returns: dict {Primary Fuel: Electricity production desired with CCS}
        """
        result = {}
        for fuel in self.elec_energy_supply_capacity:
            capacity = self.elec_energy_supply_capacity.get(fuel, 0)
            desired_util = self.elec_desired_utilization_with_ccs.get(fuel, 0)
            policy_adj = self.policy_adjustment_to_utilization.get(fuel, 0)
            ccs_frac = self.ccs_fraction_in_elec_capacity.get(fuel, 0)
            utilization = self.utilization_of_ccs_equipment.get(fuel, 0)

            result[fuel] = (capacity *
                            desired_util *
                            policy_adj *
                            ccs_frac *
                            utilization)
        return result


# Sample (simplified) inputs
total_ccs_capture_by_fuel_tons_dict = {
    'Coal': 1e9,
    'Gas': 5e8
}
elec_desired_utilization_with_ccs = {
    'Coal': 0.85,
    'Gas': 0.9
}
policy_adjustment_to_utilization = {
    'Coal': 1.0,
    'Gas': 0.95
}
ccs_fraction_in_elec_capacity = {
    'Coal': 0.6,
    'Gas': 0.5
}
utilization_of_ccs_equipment = {
    'Coal': 0.9,
    'Gas': 0.92
}
elec_energy_supply_capacity = {
    'Coal': 1000,  # GW or some consistent unit
    'Gas': 800
}

ccs = CarbonCaptureExperience(
    ccs_experience_init_multiplier=1.0,
    ccs_reference_experience=100,
    total_ccs_capture_by_fuel_tons_dict=total_ccs_capture_by_fuel_tons_dict,
    tons_co2_per_gton_co2=1e9,
    elec_desired_utilization_with_ccs=elec_desired_utilization_with_ccs,
    policy_adjustment_to_utilization=policy_adjustment_to_utilization,
    ccs_fraction_in_elec_capacity=ccs_fraction_in_elec_capacity,
    utilization_of_ccs_equipment=utilization_of_ccs_equipment,
    elec_energy_supply_capacity=elec_energy_supply_capacity
)

print(ccs.get_electricity_production_desired_with_ccs())
print(f"Carbon Capture Experience: {ccs.get_stock_value()}")


{'Coal': 459.0, 'Gas': 314.64}
Carbon Capture Experience: 100.0


CO2 Transport Capacity Stock (works)

In [ ]:
class CO2TransportCapacity:
    def __init__(self, ccs_capacity_lifetime=20):
        """
        Initializes the CO2 Transport Capacity stock.
        :param ccs_capacity_lifetime: The lifetime of CO2 transport infrastructure in years (default 20).
        """
        self.ccs_capacity_lifetime = ccs_capacity_lifetime
        self.co2_transport_capacity = 0  # Initial stock value
        self.effective_co2_transport_construction_time = 1  # Placeholder, to be replaced with equation later
        self.co2_transport_capacity_in_construction = 0  # To be defined dynamically

    def co2_transport_capacity_completions(self):
        """Calculates the rate at which new CO2 transport capacity becomes operational."""
        return self.co2_transport_capacity_in_construction / self.effective_co2_transport_construction_time if self.effective_co2_transport_construction_time > 0 else 0

    def co2_transport_retirement(self):
        """Calculates the rate at which CO2 transport infrastructure is retired."""
        return self.co2_transport_capacity / self.ccs_capacity_lifetime if self.ccs_capacity_lifetime > 0 else 0

    def update_stock(self, year):
        """Updates the CO2 transport capacity stock based on inflows and outflows."""
        completions = self.co2_transport_capacity_completions()
        retirements = self.co2_transport_retirement()
        self.co2_transport_capacity += completions - retirements
        print(f"Year {year}: CO2 Transport Capacity = {self.co2_transport_capacity:.2f} Mt")

    def get_stock_value(self):
        """Returns the current CO2 transport capacity."""
        return self.co2_transport_capacity

# Example usage

# Initialize CO2 Transport Capacity stock
co2_transport = CO2TransportCapacity()

# Set initial values
co2_transport.effective_co2_transport_construction_time = 5  # Placeholder value
co2_transport.co2_transport_capacity_in_construction = 100  # Placeholder value in Mt

# Simulate updates over 10 years
for year in range(2025, 2035):
    co2_transport.update_stock(year)


Year 2025: CO2 Transport Capacity = 20.00 Mt
Year 2026: CO2 Transport Capacity = 39.00 Mt
Year 2027: CO2 Transport Capacity = 57.05 Mt
Year 2028: CO2 Transport Capacity = 74.20 Mt
Year 2029: CO2 Transport Capacity = 90.49 Mt
Year 2030: CO2 Transport Capacity = 105.96 Mt
Year 2031: CO2 Transport Capacity = 120.67 Mt
Year 2032: CO2 Transport Capacity = 134.63 Mt
Year 2033: CO2 Transport Capacity = 147.90 Mt
Year 2034: CO2 Transport Capacity = 160.51 Mt


CO2 transport capacity in construction Stock (works)

In [ ]:
class CO2TransportCapacityInConstruction:
    def __init__(self, normal_ccs_completion_time=15):
        """
        Initializes the CO2 Transport Capacity in Construction stock.
        :param normal_ccs_completion_time: Time required to fully complete CCS infrastructure (default 15 years).
        """
        self.normal_ccs_completion_time = normal_ccs_completion_time
        self.co2_transport_capacity_in_construction = 0  # Initial stock value
        self.co2_transport_capacity_in_development = 0  # Will be set dynamically
        self.effective_co2_transport_construction_time = 1  # Placeholder, will be updated later

    def ccs_development_time(self):
        """Calculates CCS development time as half of the normal completion time."""
        return self.normal_ccs_completion_time / 2

    def co2_transport_capacity_starts(self):
        """Calculates the rate at which new CO2 transport projects begin construction."""
        return self.co2_transport_capacity_in_development / self.ccs_development_time() if self.ccs_development_time() > 0 else 0

    def co2_transport_capacity_completions(self):
        """Calculates the rate at which CO2 transport capacity completes construction."""
        return self.co2_transport_capacity_in_construction / self.effective_co2_transport_construction_time if self.effective_co2_transport_construction_time > 0 else 0

    def update_stock(self, year):
        """Updates the CO2 transport capacity in construction stock."""
        starts = self.co2_transport_capacity_starts()
        completions = self.co2_transport_capacity_completions()
        self.co2_transport_capacity_in_construction += starts - completions
        print(f"Year {year}: CO2 Transport Capacity in Construction = {self.co2_transport_capacity_in_construction:.2f} Mt")

    def get_stock_value(self):
        """Returns the current CO2 transport capacity in construction."""
        return self.co2_transport_capacity_in_construction

# Example usage

# Initialize the CO2 Transport Capacity in Construction stock
co2_transport_construction = CO2TransportCapacityInConstruction()

# Set initial values
co2_transport_construction.co2_transport_capacity_in_development = 50  # Placeholder value
co2_transport_construction.effective_co2_transport_construction_time = 5  # Placeholder value

# Simulate updates over 10 years
for year in range(2025, 2035):
    co2_transport_construction.update_stock(year)

Year 2025: CO2 Transport Capacity in Construction = 6.67 Mt
Year 2026: CO2 Transport Capacity in Construction = 12.00 Mt
Year 2027: CO2 Transport Capacity in Construction = 16.27 Mt
Year 2028: CO2 Transport Capacity in Construction = 19.68 Mt
Year 2029: CO2 Transport Capacity in Construction = 22.41 Mt
Year 2030: CO2 Transport Capacity in Construction = 24.60 Mt
Year 2031: CO2 Transport Capacity in Construction = 26.34 Mt
Year 2032: CO2 Transport Capacity in Construction = 27.74 Mt
Year 2033: CO2 Transport Capacity in Construction = 28.86 Mt
Year 2034: CO2 Transport Capacity in Construction = 29.75 Mt


CO2 Transport Capacity in Development Stock (works)

In [ ]:
class CO2TransportCapacityInDevelopment:
    def __init__(self, normal_ccs_completion_time=15, co2_transport_capacity_sl_adjustment_time=2):
        """
        Initializes the CO2 Transport Capacity in Development stock.
        :param normal_ccs_completion_time: Time required to fully complete CCS infrastructure (default 15 years).
        :param co2_transport_capacity_sl_adjustment_time: Time to adjust transport capacity based on supply chain limits (=2 in Enroads).
        """
        self.normal_ccs_completion_time = normal_ccs_completion_time
        self.co2_transport_capacity_sl_adjustment_time = co2_transport_capacity_sl_adjustment_time

        # Initial stock/variable values
        self.co2_transport_capacity_in_development = 0
        self.co2_transport_capacity_on_order = 0
        self.desired_co2_transport_capacity_on_order = 0
        self.desired_ccs_transport_capacity_completions = 0

    def ccs_development_time(self):
        """Calculates CCS development time as half of the normal completion time."""
        return self.normal_ccs_completion_time / 2

    def desired_co2_transport_orders(self):
        """Calculates the desired CO2 transport orders."""
        adjustment_term = (self.desired_co2_transport_capacity_on_order - self.co2_transport_capacity_on_order) / self.co2_transport_capacity_sl_adjustment_time
        return max(0, self.desired_ccs_transport_capacity_completions + adjustment_term)

    def co2_transport_capacity_orders(self):
        """Calculates the rate at which new CO2 transport projects are ordered."""
        return self.desired_co2_transport_orders()

    def co2_transport_capacity_starts(self):
        """Calculates the rate at which CO2 transport projects move from development to construction."""
        return self.co2_transport_capacity_in_development / self.ccs_development_time() if self.ccs_development_time() > 0 else 0

    def update_stock(self, year):
        """Updates the CO2 transport capacity in development stock."""
        orders = self.co2_transport_capacity_orders()
        starts = self.co2_transport_capacity_starts()
        self.co2_transport_capacity_in_development += orders - starts
        print(f"Year {year}: CO2 Transport Capacity in Development = {self.co2_transport_capacity_in_development:.2f} Mt")

    def get_stock_value(self):
        """Returns the current CO2 transport capacity in development."""
        return self.co2_transport_capacity_in_development

# Example Usage
# Initialize the CO2 Transport Capacity in Development stock
co2_transport_development = CO2TransportCapacityInDevelopment()

# Set initial values
co2_transport_development.co2_transport_capacity_on_order = 30  # Placeholder value
co2_transport_development.desired_co2_transport_capacity_on_order = 40  # Placeholder value
co2_transport_development.desired_ccs_transport_capacity_completions = 10  # Placeholder value

# Simulate updates over 10 years
for year in range(2025, 2035):
    co2_transport_development.update_stock(year)


Year 2025: CO2 Transport Capacity in Development = 15.00 Mt
Year 2026: CO2 Transport Capacity in Development = 28.00 Mt
Year 2027: CO2 Transport Capacity in Development = 39.27 Mt
Year 2028: CO2 Transport Capacity in Development = 49.03 Mt
Year 2029: CO2 Transport Capacity in Development = 57.49 Mt
Year 2030: CO2 Transport Capacity in Development = 64.83 Mt
Year 2031: CO2 Transport Capacity in Development = 71.18 Mt
Year 2032: CO2 Transport Capacity in Development = 76.69 Mt
Year 2033: CO2 Transport Capacity in Development = 81.47 Mt
Year 2034: CO2 Transport Capacity in Development = 85.60 Mt


Cumulative C stored by CCS Stock (works)

In [ ]:
class CumulativeCStoredByCCS:
    def __init__(self, loss_rate_from_ccs_by_path, fraction_init_c_removed_by_ccs_path, co2_captured_by_ccs_by_fuel):
        """
        Initializes the Cumulative C stored by CCS stock.

        Parameters:
        - loss_rate_from_ccs_by_path (dict): Loss rate for CCS by fuel type.
        - fraction_init_c_removed_by_ccs_path (dict): Fraction of initial C removed by CCS path for each fuel type.
        - co2_captured_by_ccs_by_fuel (dict): Function or value for CO2 captured by CCS per fuel type.
        """
        self.loss_rate_from_ccs_by_path = loss_rate_from_ccs_by_path
        self.fraction_init_c_removed_by_ccs_path = fraction_init_c_removed_by_ccs_path
        self.co2_captured_by_ccs_by_fuel = co2_captured_by_ccs_by_fuel
        self.co2_per_c = 3.66
        self.init_c_removed = 0  # Initial C removed

        # Initialize Cumulative C stored by CCS for each fuel type
        self.cumulative_c_stored = {
            fuel: self.init_c_removed * self.fraction_init_c_removed_by_ccs_path.get(fuel, 0)
            for fuel in self.fraction_init_c_removed_by_ccs_path
        }

    def c_storage_by_ccs(self):
        """Calculates carbon storage by CCS for each fuel type."""
        return {
            fuel: self.co2_captured_by_ccs_by_fuel.get(fuel, 0) / self.co2_per_c
            for fuel in self.co2_captured_by_ccs_by_fuel
        }

    def emissions_from_storage_by_ccs(self):
        """Calculates emissions from storage by CCS for each fuel type."""
        return {
            fuel: (self.cumulative_c_stored.get(fuel, 0) * self.loss_rate_from_ccs_by_path.get(fuel, 0)) / 100
            for fuel in self.loss_rate_from_ccs_by_path
        }

    def update_stock(self, dt):
        """Updates the cumulative carbon stored by CCS."""
        c_storage = self.c_storage_by_ccs()
        emissions = self.emissions_from_storage_by_ccs()

        for fuel in self.cumulative_c_stored:
            self.cumulative_c_stored[fuel] += (c_storage.get(fuel, 0) - emissions.get(fuel, 0)) * dt

# Example Usage

# Example fuel types
fuel_types = ["Coal", "Oil", "Gas", "Biomass"]

# Example input values
loss_rate_from_ccs_by_path = {"Coal": 1, "Oil": 2, "Gas": 0.5, "Biomass": 1.5}  # % loss per year
fraction_init_c_removed_by_ccs_path = {"Coal": 0.5, "Oil": 0, "Gas": 0.5, "Biomass": 0}
co2_captured_by_ccs_by_fuel = {"Coal": 100, "Oil": 50, "Gas": 80, "Biomass": 20}  # in Gt CO2

# Create instance of the stock
ccs_stock = CumulativeCStoredByCCS(
    loss_rate_from_ccs_by_path,
    fraction_init_c_removed_by_ccs_path,
    co2_captured_by_ccs_by_fuel
)

# Run simulation for 10 years with a time step of 1 year
time_steps = 10
dt = 1  # 1 year per step

print("Year | Coal  | Oil  | Gas  | Biomass")
print("-----------------------------------")
for year in range(1, time_steps + 1):
    ccs_stock.update_stock(dt)
    stored = ccs_stock.cumulative_c_stored
    print(f"{year:4} | {stored['Coal']:.2f} | {stored['Oil']:.2f} | {stored['Gas']:.2f} | {stored['Biomass']:.2f}")


Year | Coal  | Oil  | Gas  | Biomass
-----------------------------------
   1 | 27.32 | 13.66 | 21.86 | 5.46
   2 | 54.37 | 27.05 | 43.61 | 10.85
   3 | 81.15 | 40.17 | 65.25 | 16.15
   4 | 107.66 | 53.03 | 86.78 | 21.37
   5 | 133.91 | 65.63 | 108.20 | 26.51
   6 | 159.89 | 77.98 | 129.52 | 31.58
   7 | 185.61 | 90.08 | 150.73 | 36.57
   8 | 211.08 | 101.94 | 171.83 | 41.49
   9 | 236.29 | 113.56 | 192.83 | 46.33
  10 | 261.25 | 124.95 | 213.73 | 51.10


Cumulative C stored by nonAF CDR Stock (works)

In [ ]:
from typing_extensions import Self
class CumulativeCStoredByNonAFCDR:
    def __init__(self,
                 loss_rate_from_cdr_by_type,
                 fraction_init_c_removed_by_cdr_type,
                 c_removal_rate_for_nonaf_cdr,
                 c_in_atmosphere_func,
                 nonaf_cdr):
        """
        Initializes the Cumulative C Stored by nonAF CDR stock.

        :param loss_rate_from_cdr_by_type: Dict of loss rates for different sequestration types (% per year)
        :param fraction_init_c_removed_by_cdr_type: Dict of initial C fractions removed by sequestration type
        :param c_removal_rate_for_nonaf_cdr: Dict of removal rates for different sequestration types
        :param c_in_atmosphere_func: Function to retrieve current C in atmosphere (external stock)
        :param nonaf_cdr: List of sequestration types
        """
        self.loss_rate_from_cdr_by_type = loss_rate_from_cdr_by_type
        self.fraction_init_c_removed_by_cdr_type = {
            "Direct Air Capture": 0,
            "BECCS": 0,
            "Enhanced Weathering": 0,
            "Ocean Alkalinity": 0,
            "Biochar": 0
        }  # Constants defined by enroads ^
        self.c_removal_rate_for_nonaf_cdr = c_removal_rate_for_nonaf_cdr
        self.c_in_atmosphere_func = c_in_atmosphere_func
        self.nonaf_cdr = nonaf_cdr

        self.minimum_time_for_c_cycle_changes = 0.125
        self.init_c_removed = 0

        # Initialize the cumulative stored carbon stock
        self.cumulative_c_stored = {cdr_type: self.init_c_removed * self.fraction_init_c_removed_by_cdr_type.get(cdr_type, 0) for cdr_type in nonaf_cdr}
        # self.cumulative_c_stored = self.init_c_removed * self.fraction_init_c_removed_by_cdr_type

    def c_removal_by_nonaf_cdr(self):
        """Calculates the carbon removal by nonAF CDR for each sequestration type."""
        c_in_atmosphere = self.c_in_atmosphere_func()
        num_types = len(self.nonaf_cdr)  # Equivalent to ELMCOUNT(NonAF CDR)

        return {
            cdr_type: min(
                self.c_removal_rate_for_nonaf_cdr[cdr_type],
                c_in_atmosphere / num_types / self.minimum_time_for_c_cycle_changes
            )
            for cdr_type in self.nonaf_cdr
        }

    def emissions_from_storage_by_nonaf_cdr(self):
        """Calculates emissions from stored carbon for each sequestration type."""
        return {
            cdr_type: self.cumulative_c_stored[cdr_type] * self.loss_rate_from_cdr_by_type[cdr_type] / 100
            for cdr_type in self.nonaf_cdr
        }

    def update_stock(self, dt):
        """Updates the cumulative stored carbon stock over a time step dt."""
        c_removal = self.c_removal_by_nonaf_cdr()
        emissions = self.emissions_from_storage_by_nonaf_cdr()

        for cdr_type in self.nonaf_cdr:
            self.cumulative_c_stored[cdr_type] += (c_removal[cdr_type] - emissions[cdr_type]) * dt

# Example Usage (CHANGE VARIABLE NAMES)
# Define nonaf cdr types
nonaf_cdr = ["Direct Air Capture", "BECCS", "Enhanced Weathering", "Ocean Alkalinity", "Biochar"]

# Define fraction of initial c removed from each cdr type
fraction_init_c_removed_by_cdr_type = {
    "Direct Air Capture": 0,
    "BECCS": 0,
    "Enhanced Weathering": 0,
    "Ocean Alkalinity": 0,
    "Biochar": 0
}

# Initialize loss rates and removal rates for each sequestration type (example values)
loss_rate_from_cdr_by_type = {
    "Direct Air Capture": 0.01,
    "BECCS": 0.02,
    "Enhanced Weathering": 0.005,
    "Ocean Alkalinity": 0.003,
    "Biochar": 0.001
}

c_removal_rate_for_nonaf_cdr = {
    "Direct Air Capture": 10,
    "BECCS": 8,
    "Enhanced Weathering": 12,
    "Ocean Alkalinity": 15,
    "Biochar": 5
}

# Function to simulate C in the atmosphere (Placeholder)
def mock_c_in_atmosphere():
    return 800  # Example value

# Create instances for each sequestration type
cdr_stocks = {
    cdr: CumulativeCStoredByNonAFCDR(
        loss_rate_from_cdr_by_type, fraction_init_c_removed_by_cdr_type,
        c_removal_rate_for_nonaf_cdr, mock_c_in_atmosphere, nonaf_cdr
    )
    for cdr in nonaf_cdr
}

# Simulate over time
time_steps = 10
for t in range(time_steps):
    print(f"Time Step {t+1}:")
    for sequestration, cdr_stock in cdr_stocks.items():
        cdr_stock.update_stock(1)  # Assuming time step of 1 year
        stored_value = cdr_stock.cumulative_c_stored[sequestration]
        print(f"  {sequestration}: {stored_value:.2f} GtC stored")

    print("-" * 50)

Time Step 1:
  Direct Air Capture: 10.00 GtC stored
  BECCS: 8.00 GtC stored
  Enhanced Weathering: 12.00 GtC stored
  Ocean Alkalinity: 15.00 GtC stored
  Biochar: 5.00 GtC stored
--------------------------------------------------
Time Step 2:
  Direct Air Capture: 20.00 GtC stored
  BECCS: 16.00 GtC stored
  Enhanced Weathering: 24.00 GtC stored
  Ocean Alkalinity: 30.00 GtC stored
  Biochar: 10.00 GtC stored
--------------------------------------------------
Time Step 3:
  Direct Air Capture: 30.00 GtC stored
  BECCS: 24.00 GtC stored
  Enhanced Weathering: 36.00 GtC stored
  Ocean Alkalinity: 45.00 GtC stored
  Biochar: 15.00 GtC stored
--------------------------------------------------
Time Step 4:
  Direct Air Capture: 39.99 GtC stored
  BECCS: 31.99 GtC stored
  Enhanced Weathering: 48.00 GtC stored
  Ocean Alkalinity: 60.00 GtC stored
  Biochar: 20.00 GtC stored
--------------------------------------------------
Time Step 5:
  Direct Air Capture: 49.99 GtC stored
  BECCS: 39.98

Cumulative CO2 flow to Storage Stock (works)

In [ ]:
class CumulativeCO2FlowToStorage:
    def __init__(self):
        # Initial value of the stock
        self.cumulative_co2_flow_to_storage = 0.0

    def update(self, dt, co2_captured_by_fuel, gross_cdr_by_type):
        """
        Updates the stock using integration.

        Parameters:
        - dt: Time step
        - co2_captured_by_fuel: Dictionary containing CO2 captured by CCS for different fuels (PBio, PCoal, PGas, POil)
        - gross_cdr_by_type: CO2 removed by Direct Air Capture (DAC)
        """
        # Compute total flow to storage
        total_flow_to_storage = (
            co2_captured_by_fuel.get("PBio", 0) +
            co2_captured_by_fuel.get("PCoal", 0) +
            gross_cdr_by_type.get("Direct Air Capture", 0) +
            co2_captured_by_fuel.get("PGas", 0) +
            co2_captured_by_fuel.get("POil", 0)
        )

        # Integration step
        self.cumulative_co2_flow_to_storage += total_flow_to_storage * dt

    def get_cumulative_co2_flow_to_storage(self):
        """Returns the total CO2 flow to storage."""
        return self.cumulative_co2_flow_to_storage


# Example usage
# Initialize the stock
co2_storage = CumulativeCO2FlowToStorage()

# Define time step
dt = 1  # 1 year (or any other time unit)

# Mock data for CO2 captured by CCS by fuel (values in gigatonnes)
co2_captured_by_fuel = {
    "PBio": 0.5,   # BECCS
    "PCoal": 1.2,  # Coal CCS
    "PGas": 0.8,   # Gas CCS
    "POil": 0.3    # Oil CCS
}

# Mock data for Gross CDR by Direct Air Capture (in gigatonnes)
gross_cdr_by_type = {
    "Direct Air Capture": 0.7
}

# Simulate for 10 years
for year in range(10):
    co2_storage.update(dt, co2_captured_by_fuel, gross_cdr_by_type)
    print(f"Year {year+1}: Cumulative CO2 Flow to Storage = {co2_storage.get_cumulative_co2_flow_to_storage():.2f} Gt")


Year 1: Cumulative CO2 Flow to Storage = 3.50 Gt
Year 2: Cumulative CO2 Flow to Storage = 7.00 Gt
Year 3: Cumulative CO2 Flow to Storage = 10.50 Gt
Year 4: Cumulative CO2 Flow to Storage = 14.00 Gt
Year 5: Cumulative CO2 Flow to Storage = 17.50 Gt
Year 6: Cumulative CO2 Flow to Storage = 21.00 Gt
Year 7: Cumulative CO2 Flow to Storage = 24.50 Gt
Year 8: Cumulative CO2 Flow to Storage = 28.00 Gt
Year 9: Cumulative CO2 Flow to Storage = 31.50 Gt
Year 10: Cumulative CO2 Flow to Storage = 35.00 Gt


Cumulative net removal by nonAF CDR Stock (works)

In [ ]:
class CumulativeNetRemovalByNonAFCDR:
    def __init__(self, sequestration_types, loss_rate_from_cdr_by_type, co2_per_c):
        self.sequestration_types = sequestration_types  # List of sequestration types
        self.loss_rate_from_cdr_by_type = loss_rate_from_cdr_by_type  # Loss rate dictionary
        self.co2_per_c = co2_per_c  # Conversion factor (3.66)

        # Initialize stocks
        self.cumulative_net_removal = {cdr_type: 0 for cdr_type in sequestration_types}
        self.cumulative_c_stored = {cdr_type: 0 for cdr_type in sequestration_types}

    def update(self, removal_rates, c_in_atmosphere, min_time_for_c_cycle_changes, emissions_due_to_dac, emissions_due_to_beccs, additional_beccs_emissions, dt=1):
        net_removal = {}

        for cdr_type in self.sequestration_types:
            # Carbon Removal
            c_removal = min(removal_rates[cdr_type], c_in_atmosphere / len(self.sequestration_types) / min_time_for_c_cycle_changes)

            # Emissions from storage
            emissions_from_storage = self.cumulative_c_stored[cdr_type] * self.loss_rate_from_cdr_by_type[cdr_type] / 100

            # Additional emissions from CDR
            if cdr_type == "Direct air capture":
                additional_emissions = emissions_due_to_dac / self.co2_per_c
            elif cdr_type == "BECCS":
                additional_emissions = (emissions_due_to_beccs + additional_beccs_emissions) / self.co2_per_c
            else:
                additional_emissions = 0  # Other types have zero additional emissions

            # Net Carbon Removal Calculation
            net_removal[cdr_type] = c_removal - emissions_from_storage - additional_emissions

            # Update cumulative net removal
            self.cumulative_net_removal[cdr_type] += net_removal[cdr_type] * dt

        return self.cumulative_net_removal

# Example Usage
sequestration_types = ["Direct air capture", "Mineralization", "Biochar", "BECCS", "Afforestation", "Agricultural soil carbon"]
loss_rate_from_cdr_by_type = {"Direct air capture": 5, "Mineralization": 2, "Biochar": 1, "BECCS": 3, "Afforestation": 1, "Agricultural soil carbon": 1}
co2_per_c = 3.66

model = CumulativeNetRemovalByNonAFCDR(sequestration_types, loss_rate_from_cdr_by_type, co2_per_c)

removal_rates = {"Direct air capture": 100, "Mineralization": 50, "Biochar": 20, "BECCS": 80, "Afforestation": 30, "Agricultural soil carbon": 25}
c_in_atmosphere = 10000
min_time_for_c_cycle_changes = 0.125
emissions_due_to_dac = 500
emissions_due_to_beccs = 300
additional_beccs_emissions = 200

for _ in range(10):  # Simulating 10 time steps
    net_removal = model.update(removal_rates, c_in_atmosphere, min_time_for_c_cycle_changes, emissions_due_to_dac, emissions_due_to_beccs, additional_beccs_emissions)
    print(net_removal)


{'Direct air capture': -36.61202185792348, 'Mineralization': 50.0, 'Biochar': 20.0, 'BECCS': -56.61202185792348, 'Afforestation': 30.0, 'Agricultural soil carbon': 25.0}
{'Direct air capture': -73.22404371584696, 'Mineralization': 100.0, 'Biochar': 40.0, 'BECCS': -113.22404371584696, 'Afforestation': 60.0, 'Agricultural soil carbon': 50.0}
{'Direct air capture': -109.83606557377044, 'Mineralization': 150.0, 'Biochar': 60.0, 'BECCS': -169.83606557377044, 'Afforestation': 90.0, 'Agricultural soil carbon': 75.0}
{'Direct air capture': -146.44808743169392, 'Mineralization': 200.0, 'Biochar': 80.0, 'BECCS': -226.44808743169392, 'Afforestation': 120.0, 'Agricultural soil carbon': 100.0}
{'Direct air capture': -183.0601092896174, 'Mineralization': 250.0, 'Biochar': 100.0, 'BECCS': -283.0601092896174, 'Afforestation': 150.0, 'Agricultural soil carbon': 125.0}
{'Direct air capture': -219.67213114754088, 'Mineralization': 300.0, 'Biochar': 120.0, 'BECCS': -339.6721311475409, 'Afforestation': 180

DAC capacity in construction

In [ ]:
class DACCapacityInConstruction:
    def __init__(self, dac_capacity_in_development, dac_development_time, effective_dac_construction_time):
        """
        Initializes the DAC capacity in construction stock.

        :param dac_capacity_in_development: Function that returns the DAC capacity in development (tons CO2/year)
        :param dac_development_time: Function that returns the DAC development time (years)
        :param effective_dac_construction_time: Function that returns the effective DAC construction time (years)
        """
        self.dac_capacity_in_development = dac_capacity_in_development
        self.dac_development_time = dac_development_time
        self.effective_dac_construction_time = effective_dac_construction_time

        # Initial DAC capacity in construction
        self.dac_capacity_in_construction = 0

    def dac_capacity_starts(self):
        """Calculates the rate of DAC capacity starting construction."""
        return self.dac_capacity_in_development() / self.dac_development_time()

    def dac_capture_completions(self):
        """Calculates the completion rate of DAC capture infrastructure."""
        return self.dac_capacity_in_construction / self.effective_dac_construction_time()

    def update_stock(self, dt):
        """Updates the DAC capacity in construction over a time step dt."""
        new_starts = self.dac_capacity_starts()
        completions = self.dac_capture_completions()

        self.dac_capacity_in_construction += (new_starts - completions) * dt

# Example Usage

def mock_dac_capacity_in_development():
    return 100  # Example value in tons CO2/year

def mock_dac_development_time():
    return 7.5  # Example value in years

def mock_effective_dac_construction_time():
    return 8.0  # Example value in years

# Create an instance of DAC capacity in construction
dac_construction = DACCapacityInConstruction(mock_dac_capacity_in_development, mock_dac_development_time, mock_effective_dac_construction_time)

# Simulate over time
time_steps = 10
for t in range(time_steps):
    dac_construction.update_stock(1)  # Assuming time step of 1 year
    print(f"Time Step {t+1}: DAC Capacity in Construction: {dac_construction.dac_capacity_in_construction:.2f} tons CO2/year")


Time Step 1: DAC Capacity in Construction: 13.33 tons CO2/year
Time Step 2: DAC Capacity in Construction: 25.00 tons CO2/year
Time Step 3: DAC Capacity in Construction: 35.21 tons CO2/year
Time Step 4: DAC Capacity in Construction: 44.14 tons CO2/year
Time Step 5: DAC Capacity in Construction: 51.96 tons CO2/year
Time Step 6: DAC Capacity in Construction: 58.80 tons CO2/year
Time Step 7: DAC Capacity in Construction: 64.78 tons CO2/year
Time Step 8: DAC Capacity in Construction: 70.02 tons CO2/year
Time Step 9: DAC Capacity in Construction: 74.60 tons CO2/year
Time Step 10: DAC Capacity in Construction: 78.61 tons CO2/year


DAC capacity in development

In [ ]:
class DACCapacityInDevelopment:
    def __init__(self,
                 dac_capture_retirement,
                 desired_dac_capacity,
                 dac_capture_capacity,
                 dac_capacity,
                 dac_capacity_sl_adjustment_time,
                 dac_development_time,
                 dac_capacity_in_development_func):
        """
        Initializes the DAC capacity in development stock.

        :param dac_capture_retirement: Function that returns retirement rate of DAC capture infrastructure
        :param desired_dac_capacity: Function that returns target DAC capture capacity
        :param dac_capture_capacity: Function that returns current DAC capture capacity
        :param dac_capacity: Function that returns DAC capacity on order (derived from completions * normal DAC completion time)
        :param dac_capacity_sl_adjustment_time: Scalar value (years)
        :param dac_development_time: Scalar value (years)
        :param dac_capacity_in_development_func: Function to return the current DAC capacity in development
        """
        self.dac_capture_retirement = dac_capture_retirement
        self.desired_dac_capacity = desired_dac_capacity
        self.dac_capture_capacity = dac_capture_capacity
        self.dac_capacity = dac_capacity
        self.dac_capacity_sl_adjustment_time = dac_capacity_sl_adjustment_time
        self.dac_development_time = dac_development_time
        self.dac_capacity_in_development = 0
        self.dac_capacity_in_development_func = dac_capacity_in_development_func

    def desired_dac_capacity_completions(self):
        return self.dac_capture_retirement() + (
            self.desired_dac_capacity() - self.dac_capture_capacity()
        ) / self.dac_capacity_adjustment_time()

    def dac_capacity_adjustment_time(self):
        # Assuming same as DAC SL adjustment time for this context
        return self.dac_capacity_sl_adjustment_time

    def desired_dac_orders(self):
        return max(
            0,
            self.desired_dac_capacity_completions()
            + (
                self.desired_dac_capacity_on_order() - self.dac_capacity()
            ) / self.dac_capacity_sl_adjustment_time
        )

    def desired_dac_capacity_on_order(self):
        return self.desired_dac_capacity_completions() * self.normal_dac_completion_time()

    def normal_dac_completion_time(self):
        return self.dac_development_time + self.dac_construction_time()

    def dac_construction_time(self):
        # Placeholder constant; should be provided or calculated
        return 7.5

    def dac_capacity_orders(self):
        return self.desired_dac_orders()

    def dac_capacity_starts(self):
        return self.dac_capacity_in_development_func() / self.dac_development_time

    def update_stock(self, dt):
        orders = self.dac_capacity_orders()
        starts = self.dac_capacity_starts()

        self.dac_capacity_in_development += (orders - starts) * dt

# Example usage

def mock_retirement(): return 10

def mock_desired_capacity(): return 1000

def mock_current_capacity(): return 400

def mock_on_order(): return 800

def mock_current_dev(): return 300

# Create instance
dac_dev = DACCapacityInDevelopment(
    dac_capture_retirement=mock_retirement,
    desired_dac_capacity=mock_desired_capacity,
    dac_capture_capacity=mock_current_capacity,
    dac_capacity=mock_on_order,
    dac_capacity_sl_adjustment_time=2,
    dac_development_time=7.5,
    dac_capacity_in_development_func=mock_current_dev
)

# Simulate over 10 time steps
for t in range(10):
    dac_dev.update_stock(1)
    print(f"Time Step {t+1}: DAC Capacity in Development: {dac_dev.dac_capacity_in_development:.2f} tons CO2/year")

Time Step 1: DAC Capacity in Development: 2195.00 tons CO2/year
Time Step 2: DAC Capacity in Development: 4390.00 tons CO2/year
Time Step 3: DAC Capacity in Development: 6585.00 tons CO2/year
Time Step 4: DAC Capacity in Development: 8780.00 tons CO2/year
Time Step 5: DAC Capacity in Development: 10975.00 tons CO2/year
Time Step 6: DAC Capacity in Development: 13170.00 tons CO2/year
Time Step 7: DAC Capacity in Development: 15365.00 tons CO2/year
Time Step 8: DAC Capacity in Development: 17560.00 tons CO2/year
Time Step 9: DAC Capacity in Development: 19755.00 tons CO2/year
Time Step 10: DAC Capacity in Development: 21950.00 tons CO2/year


DAC capture capacity

In [ ]:
class DACCaptureCapacity:
    def __init__(self,
                 dac_capacity_in_construction_func,
                 dac_construction_time,
                 loading_of_dac_construction_capability,
                 dac_capacity_lifetime):
        """
        Initializes the DAC capture capacity stock.

        :param dac_capacity_in_construction_func: Function returning current DAC capacity in construction (tons CO2/year)
        :param dac_construction_time: Scalar base construction time (years)
        :param loading_of_dac_construction_capability: Function returning loading factor
        :param dac_capacity_lifetime: Scalar lifetime of DAC components (years)
        """
        self.dac_capacity_in_construction_func = dac_capacity_in_construction_func
        self.dac_construction_time = dac_construction_time
        self.loading_of_dac_construction_capability = loading_of_dac_construction_capability
        self.dac_capacity_lifetime = dac_capacity_lifetime

        self.dac_capture_capacity = 0  # Initial value

    def effective_dac_construction_time(self):
        return self.dac_construction_time * max(1, self.loading_of_dac_construction_capability())

    def dac_capture_completions(self):
        return self.dac_capacity_in_construction_func() / self.effective_dac_construction_time()

    def dac_capture_retirement(self):
        return self.dac_capture_capacity / self.dac_capacity_lifetime

    def update_stock(self, dt):
        completions = self.dac_capture_completions()
        retirement = self.dac_capture_retirement()

        self.dac_capture_capacity += (completions - retirement) * dt


def mock_dac_capacity_in_construction(): return 600

def mock_loading(): return 1.0  # no overload assumed

capture = DACCaptureCapacity(
    dac_capacity_in_construction_func=mock_dac_capacity_in_construction,
    dac_construction_time=7.5,
    loading_of_dac_construction_capability=mock_loading,
    dac_capacity_lifetime=20
)

for t in range(10):
    capture.update_stock(1)
    print(f"Time Step {t+1}: DAC Capture Capacity: {capture.dac_capture_capacity:.2f} tons CO2/year")


Time Step 1: DAC Capture Capacity: 80.00 tons CO2/year
Time Step 2: DAC Capture Capacity: 156.00 tons CO2/year
Time Step 3: DAC Capture Capacity: 228.20 tons CO2/year
Time Step 4: DAC Capture Capacity: 296.79 tons CO2/year
Time Step 5: DAC Capture Capacity: 361.95 tons CO2/year
Time Step 6: DAC Capture Capacity: 423.85 tons CO2/year
Time Step 7: DAC Capture Capacity: 482.66 tons CO2/year
Time Step 8: DAC Capture Capacity: 538.53 tons CO2/year
Time Step 9: DAC Capture Capacity: 591.60 tons CO2/year
Time Step 10: DAC Capture Capacity: 642.02 tons CO2/year


DAC experience

In [ ]:
class DACExperience:
    def __init__(self,
                 co2_capture_rate_by_dac_tons_func,
                 tons_per_gt=1e9,
                 ccs_experience_init_multiplier=1,
                 ccs_reference_experience=2):
        """
        Initializes the DAC Experience stock.

        :param co2_capture_rate_by_dac_tons_func: Function returning CO2 capture rate by DAC in tons/year
        :param tons_per_gt: Conversion from tons to GtCO2 (default 1e9)
        :param ccs_experience_init_multiplier: Multiplier for initial experience transfer from CCS
        :param ccs_reference_experience: Reference experience for CCS in GtCO2
        """
        self.co2_capture_rate_by_dac_tons_func = co2_capture_rate_by_dac_tons_func
        self.tons_per_gt = tons_per_gt
        self.ccs_experience_init_multiplier = ccs_experience_init_multiplier
        self.ccs_reference_experience = ccs_reference_experience

        # Initial value
        self.dac_experience = ccs_experience_init_multiplier * ccs_reference_experience

    def co2_capture_rate_by_dac_gt(self):
        """Convert tons/year to GtCO2/year."""
        return self.co2_capture_rate_by_dac_tons_func() / self.tons_per_gt

    def update_stock(self, dt):
        """Update DAC experience over time step dt."""
        experience_gain = self.co2_capture_rate_by_dac_gt()
        self.dac_experience += experience_gain * dt


def mock_capture_tons():
    return 1.2e9  # 1.2 GtCO2/year


dac_exp = DACExperience(
    co2_capture_rate_by_dac_tons_func=mock_capture_tons,
    tons_per_gt=1e9,
    ccs_experience_init_multiplier=1,
    ccs_reference_experience=2
)

for t in range(10):
    dac_exp.update_stock(1)
    print(f"Time Step {t+1}: DAC Experience = {dac_exp.dac_experience:.2f} GtCO2")

Time Step 1: DAC Experience = 3.20 GtCO2
Time Step 2: DAC Experience = 4.40 GtCO2
Time Step 3: DAC Experience = 5.60 GtCO2
Time Step 4: DAC Experience = 6.80 GtCO2
Time Step 5: DAC Experience = 8.00 GtCO2
Time Step 6: DAC Experience = 9.20 GtCO2
Time Step 7: DAC Experience = 10.40 GtCO2
Time Step 8: DAC Experience = 11.60 GtCO2
Time Step 9: DAC Experience = 12.80 GtCO2
Time Step 10: DAC Experience = 14.00 GtCO2


Electric CCS

In [ ]:
class ElectricCCSDevelopment:
    def __init__(self, primary_fuels, desired_orders_func, development_time):
        self.primary_fuels = primary_fuels
        self.desired_orders_func = desired_orders_func
        self.development_time = development_time
        self.electric_ccs_in_development = {fuel: 0 for fuel in primary_fuels}

    def electric_ccs_starts(self):
        return {
            fuel: self.electric_ccs_in_development[fuel] / self.development_time
            for fuel in self.primary_fuels
        }

    def update_stock(self, dt):
        starts = self.electric_ccs_starts()
        orders = self.desired_orders_func()
        for fuel in self.primary_fuels:
            self.electric_ccs_in_development[fuel] += (orders[fuel] - starts[fuel]) * dt


class ElectricCCSConstruction:
    def __init__(self, primary_fuels, construction_time_func, development_module):
        self.primary_fuels = primary_fuels
        self.construction_time_func = construction_time_func
        self.development_module = development_module
        self.electric_ccs_in_construction = {fuel: 0 for fuel in primary_fuels}

    def electric_ccs_completions(self):
        return {
            fuel: self.electric_ccs_in_construction[fuel] / self.construction_time_func()
            for fuel in self.primary_fuels
        }

    def update_stock(self, dt):
        starts = self.development_module.electric_ccs_starts()
        completions = self.electric_ccs_completions()
        for fuel in self.primary_fuels:
            self.electric_ccs_in_construction[fuel] += (starts[fuel] - completions[fuel]) * dt

if __name__ == "__main__":
    primary_fuels = ["Coal", "Gas", "Oil"]
    normal_ccs_completion_time = 15

    def mock_desired_orders():
        return {fuel: 10 for fuel in primary_fuels}  # constant order rate

    def mock_loading():
        return 1.0

    def construction_time():
        return normal_ccs_completion_time * max(1, mock_loading())

    electric_dev = ElectricCCSDevelopment(
        primary_fuels, mock_desired_orders, normal_ccs_completion_time / 2
    )
    electric_const = ElectricCCSConstruction(
        primary_fuels, construction_time, electric_dev
    )

    for t in range(10):
        electric_dev.update_stock(1)
        electric_const.update_stock(1)
        print(f"Year {t+1}:")
        for fuel in primary_fuels:
            print(f"  Electric CCS Dev [{fuel}]: {electric_dev.electric_ccs_in_development[fuel]:.2f}")
            print(f"  Electric CCS Const [{fuel}]: {electric_const.electric_ccs_in_construction[fuel]:.2f}")
        print("-")


Year 1:
  Electric CCS Dev [Coal]: 10.00
  Electric CCS Const [Coal]: 1.33
  Electric CCS Dev [Gas]: 10.00
  Electric CCS Const [Gas]: 1.33
  Electric CCS Dev [Oil]: 10.00
  Electric CCS Const [Oil]: 1.33
-
Year 2:
  Electric CCS Dev [Coal]: 18.67
  Electric CCS Const [Coal]: 3.73
  Electric CCS Dev [Gas]: 18.67
  Electric CCS Const [Gas]: 3.73
  Electric CCS Dev [Oil]: 18.67
  Electric CCS Const [Oil]: 3.73
-
Year 3:
  Electric CCS Dev [Coal]: 26.18
  Electric CCS Const [Coal]: 6.97
  Electric CCS Dev [Gas]: 26.18
  Electric CCS Const [Gas]: 6.97
  Electric CCS Dev [Oil]: 26.18
  Electric CCS Const [Oil]: 6.97
-
Year 4:
  Electric CCS Dev [Coal]: 32.69
  Electric CCS Const [Coal]: 10.87
  Electric CCS Dev [Gas]: 32.69
  Electric CCS Const [Gas]: 10.87
  Electric CCS Dev [Oil]: 32.69
  Electric CCS Const [Oil]: 10.87
-
Year 5:
  Electric CCS Dev [Coal]: 38.33
  Electric CCS Const [Coal]: 15.25
  Electric CCS Dev [Gas]: 38.33
  Electric CCS Const [Gas]: 15.25
  Electric CCS Dev [Oil]: 3

Industry CCS

In [ ]:
class IndustryCCSDevelopment:
    def __init__(self, primary_fuels, desired_orders_func, development_time):
        self.primary_fuels = primary_fuels
        self.desired_orders_func = desired_orders_func
        self.development_time = development_time
        self.industry_ccs_in_development = {fuel: 0 for fuel in primary_fuels}

    def industry_ccs_starts(self):
        return {
            fuel: self.industry_ccs_in_development[fuel] / self.development_time
            for fuel in self.primary_fuels
        }

    def update_stock(self, dt):
        starts = self.industry_ccs_starts()
        orders = self.desired_orders_func()
        for fuel in self.primary_fuels:
            self.industry_ccs_in_development[fuel] += (orders[fuel] - starts[fuel]) * dt


class IndustryCCSConstruction:
    def __init__(self, primary_fuels, construction_time_func, development_module):
        self.primary_fuels = primary_fuels
        self.construction_time_func = construction_time_func
        self.development_module = development_module
        self.industry_ccs_in_construction = {fuel: 0 for fuel in primary_fuels}

    def industry_ccs_completions(self):
        return {
            fuel: self.industry_ccs_in_construction[fuel] / self.construction_time_func()
            for fuel in self.primary_fuels
        }

    def update_stock(self, dt):
        starts = self.development_module.industry_ccs_starts()
        completions = self.industry_ccs_completions()
        for fuel in self.primary_fuels:
            self.industry_ccs_in_construction[fuel] += (starts[fuel] - completions[fuel]) * dt

if __name__ == "__main__":
    primary_fuels = ["Coal", "Gas", "Oil"]
    normal_ccs_completion_time = 15

    def mock_desired_orders():
        return {fuel: 10 for fuel in primary_fuels}  # constant order rate

    def mock_loading():
        return 1.0

    def construction_time():
        return normal_ccs_completion_time * max(1, mock_loading())

    industry_dev = IndustryCCSDevelopment(
        primary_fuels, mock_desired_orders, normal_ccs_completion_time / 2
    )
    industry_const = IndustryCCSConstruction(
        primary_fuels, construction_time, industry_dev
    )

    for t in range(10):
        industry_dev.update_stock(1)
        industry_const.update_stock(1)
        print(f"Year {t+1}:")
        for fuel in primary_fuels:
            print(f"  Industry CCS Dev [{fuel}]: {industry_dev.industry_ccs_in_development[fuel]:.2f}")
            print(f"  Industry CCS Const [{fuel}]: {industry_const.industry_ccs_in_construction[fuel]:.2f}")
        print("-")


Year 1:
  Industry CCS Dev [Coal]: 10.00
  Industry CCS Const [Coal]: 1.33
  Industry CCS Dev [Gas]: 10.00
  Industry CCS Const [Gas]: 1.33
  Industry CCS Dev [Oil]: 10.00
  Industry CCS Const [Oil]: 1.33
-
Year 2:
  Industry CCS Dev [Coal]: 18.67
  Industry CCS Const [Coal]: 3.73
  Industry CCS Dev [Gas]: 18.67
  Industry CCS Const [Gas]: 3.73
  Industry CCS Dev [Oil]: 18.67
  Industry CCS Const [Oil]: 3.73
-
Year 3:
  Industry CCS Dev [Coal]: 26.18
  Industry CCS Const [Coal]: 6.97
  Industry CCS Dev [Gas]: 26.18
  Industry CCS Const [Gas]: 6.97
  Industry CCS Dev [Oil]: 26.18
  Industry CCS Const [Oil]: 6.97
-
Year 4:
  Industry CCS Dev [Coal]: 32.69
  Industry CCS Const [Coal]: 10.87
  Industry CCS Dev [Gas]: 32.69
  Industry CCS Const [Gas]: 10.87
  Industry CCS Dev [Oil]: 32.69
  Industry CCS Const [Oil]: 10.87
-
Year 5:
  Industry CCS Dev [Coal]: 38.33
  Industry CCS Const [Coal]: 15.25
  Industry CCS Dev [Gas]: 38.33
  Industry CCS Const [Gas]: 15.25
  Industry CCS Dev [Oil]: 3